# **Setup**

In [ ]:
!pip install -qq langchain langchain-core langchain-google-genai langchain-community


In [15]:
import os

In [16]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [17]:
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY


In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3,google_api_key=GEMINI_API_KEY)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

# **Conversation buffer memory**
keeps all conversation history (big context → expensive)

In [23]:
!pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [28]:
# 1. Define where to keep memory (per-session store)
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 2. Prompt with history slot
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly assistant."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

# 3. Base chain
base_chain = prompt | llm | StrOutputParser()

# 4. Add memory wrapper
chain = RunnableWithMessageHistory(
    base_chain,
    get_session_history,                # function that manages histories
    history_messages_key="history",     # must match MessagesPlaceholder
    input_messages_key="input"          # user input field
)

# 5. Run conversation (provide session_id each time)
print(chain.invoke(
    {"input": "hello, my name is ali."},
    config={"configurable": {"session_id": "user-123"}}
))



Hello Ali! It's nice to meet you.  How can I help you today?


In [29]:
print(chain.invoke(
    {"input": "what’s my name?"},
    config={"configurable": {"session_id": "user-123"}}
))

Your name is Ali.


# **Conversation buffer window memory**
only last n exchanges (cheaper, but forgets old info).

In [30]:
# 1. Store with window control
store = {}
WINDOW_SIZE = 3  # keep last 3 messages

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    history = store[session_id]
    # trim history to last k messages
    if len(history.messages) > WINDOW_SIZE:
        history.messages = history.messages[-WINDOW_SIZE:]
    return history

# 2. Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly assistant."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

# 3. Base chain
base_chain = prompt | llm | StrOutputParser()

# 4. Memory wrapper
chain = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    history_messages_key="history",
    input_messages_key="input"
)

# 5. Run
print(chain.invoke({"input": "Hello, I am Ali."}, config={"configurable": {"session_id": "u1"}}))
print(chain.invoke({"input": "I live in Karachi."}, config={"configurable": {"session_id": "u1"}}))
print(chain.invoke({"input": "I study computer science."}, config={"configurable": {"session_id": "u1"}}))
print(chain.invoke({"input": "What’s my name and where do I live?"}, config={"configurable": {"session_id": "u1"}}))


Hello Ali! It's nice to meet you. How can I help you today?
That's great, Ali! Karachi is a vibrant city.  Is there anything specific you'd like to talk about regarding Karachi, or anything else I can help you with?
That's fantastic, Ali! Computer Science is a fascinating and rapidly evolving field.  What aspects of computer science are you most interested in?  Are you currently working on any interesting projects?  I'd love to hear more.
I don't have access to personal information about you, Ali, unless you explicitly provide it to me.  My purpose is to help and provide information, but I prioritize your privacy.  I only know what you've told me in our current conversation.


# **Summarizer memory**
keeps a running summary instead of raw history (scales better).

In [32]:
from langchain.memory import ConversationSummaryBufferMemory

# 1. Memory store
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ConversationSummaryBufferMemory(
            llm=llm,
            return_messages=True,
            max_token_limit=200
        )
    return store[session_id].chat_memory  # ✅ exposes .messages

# 2. Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly assistant."),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])

# 3. Base chain
base_chain = prompt | llm | StrOutputParser()

# 4. Memory wrapper
chain = RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    history_messages_key="history",
    input_messages_key="input"
)

# 5. Run
print(chain.invoke({"input": "Hello, I am Ali."}, config={"configurable": {"session_id": "u2"}}))
print(chain.invoke({"input": "I live in Karachi."}, config={"configurable": {"session_id": "u2"}}))
print(chain.invoke({"input": "I study computer science."}, config={"configurable": {"session_id": "u2"}}))
print(chain.invoke({"input": "What’s my name and what did I tell you earlier?"}, config={"configurable": {"session_id": "u2"}}))


/tmp/ipython-input-3651643799.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  store[session_id] = ConversationSummaryBufferMemory(


Hello Ali! It's nice to meet you. How can I help you today?
That's great, Ali! Karachi is a vibrant city.  Is there anything specific you'd like to talk about regarding Karachi, or anything else I can help you with?
That's fantastic, Ali! Computer Science is a fascinating and rapidly evolving field.  Are you working on any interesting projects right now?  Or perhaps you have questions about a particular area of computer science?  I'd be happy to chat if you'd like.
Your name is Ali, and you told me you live in Karachi and you study computer science.
